<a href="https://colab.research.google.com/github/fendy07/chatbot-AI/blob/master/DL_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chatbot Using Long Short Term Memory Algorithm**

Sumber code: [Medium - Going Merry With Tensorflow 2.0](https://medium.com/analytics-vidhya/chatbot-with-tensorflow-2-0-going-merry-2f79284a6104)

# **Import Library**

In [1]:
# Import Libraries
import json
import nltk
import time
import random
import string
import pickle
import numpy as np
import pandas as pd
from io import BytesIO
import tensorflow as tf
import IPython.display as ipd
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Model
from keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Flatten, Dense, GlobalMaxPool1D

## **Download NLTK Package**

In [2]:
# Package sentence tokenizer
nltk.download('punkt')
# Package lemmatization
nltk.download('wordnet')
# Package multilingual wordnet data
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# **Load Dataset Json**

In [3]:
# Importing the dataset
with open('data.json', encoding='utf-8') as content:
  data1 = json.load(content)

# Mendapatkan semua data ke dalam list
tags = [] # data tag
inputs = [] # data input atau pattern
responses = {} # data respon
words = [] # Data kata
classes = [] # Data Kelas atau Tag
documents = [] # Data Kalimat Dokumen
ignore_words = ['?', '!'] # Mengabaikan tanda spesial karakter
# Tambahkan data intents dalam json
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])
    # digunakan untuk pattern atau teks pertanyaan dalam json
    for pattern in intent['patterns']:
      w = nltk.word_tokenize(pattern)
      words.extend(w)
      documents.append((w, intent['tag']))
      # tambahkan ke dalam list kelas dalam data
      if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Konversi data json ke dalam dataframe
data = pd.DataFrame({"patterns":inputs, "tags":tags})

In [4]:
# Cetak data keseluruhan
data

,patterns,tags
0,hallo,greeting
1,Hai kak,greeting
2,helo,greeting
3,hai,greeting
4,halo,greeting
...,...,...
1827,Untar punya wakil rektor siapa sih?,wakil_rektor
1828,Jabatan wakil rektor di Untar dipegang siapa?,wakil_rektor
1829,Siapa yang jadi wakil rektor Untar sekarang?,wakil_rektor
1830,Informasi wakil rektor Untar siapa?,wakil_rektor


In [5]:
# Cetak data baris pertama sampai baris kelima
data.head()

,patterns,tags
0,hallo,greeting
1,Hai kak,greeting
2,helo,greeting
3,hai,greeting
4,halo,greeting


# **Preprocessing The Data**

1.   Remove Punctuations (Menghapus Punktuasi)
2.   Lematization (Lematisasi)
3.   Tokenization (Tokenisasi)
4.   Apply Padding (Padding)
5.   Encoding the Outputs (Konversi Keluaran Enkoding)

Kelima tahapan pemrosesan teks ini dijelaskan pada bagian langkah selanjutnya.

## **Remove Punctuations**

Tahapan praproses pada data teks yang pertama adalah menghapus punktuasi atau tanda baca seperti *special character* yaitu **'!'** (**tanda seru**) **','** (**tanda koma**) **'.'** (**tanda titik sebagai berhenti**) '**?**' (**tanda tanya**) dan tanda baca yang lain. Tahapan ini gunanya untuk mempermudah pemrosesan data teks yang akan kita olah.

In [6]:
# Removing Punctuations (Menghilangkan Punktuasi)
data['patterns'] = data['patterns'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['patterns'] = data['patterns'].apply(lambda wrd: ''.join(wrd))

## **Lemmatization (Lematisasi)**

In [7]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

print (len(words), "unique lemmatized words", words)

981 unique lemmatized words ['(', ')', ',', '2024/2025', '24', '4.0', '[', 'a', 'acara', 'acara-acara', 'ada', 'adain', 'administrasi', 'afternoon', 'agama', 'agar', 'aja', 'ajaran', 'ajukan', 'ak', 'akademik', 'akhir', 'akhir-akhir', 'akreditasi', 'aktif', 'alamat', 'almamater', 'almameter', 'altar', 'alumni-alumni', 'alumnus', 'alur', 'alurnya', 'ambil', 'anggota', 'angka', 'antar', 'antara', 'apa', 'apakah', 'apasih', 'aplikasi', 'arah', 'area', 'artis', 'asia', 'aspek', 'asrama', 'atas', 'atau', 'aturan', 'auditorium', 'awal', 'b', 'baca', 'badan', 'bagaimana', 'bagi', 'bagian', 'bagus', 'baik', 'bakat', 'ban-pt', 'bank', 'bantu', 'bantuan', 'banyak', 'bareng', 'barengan', 'baru', 'batal', 'batalin', 'bawa', 'bay', 'bayar', 'beasiswa', 'beberapa', 'beda', 'bedanya', 'bekerja', 'belajar', 'belajar-mengajarnya', 'belakang', 'belum', 'bem', 'bentuk', 'berada', 'beradaptasi', 'berakhir', 'berakhirnya', 'berapa', 'berbagai', 'berbasis', 'berbeda', 'berdampak', 'berdasarkan', 'berdiri', 

### **Menyortir Data Kelas Tags**

In [8]:
# sorting pada data class
classes = sorted(list(set(classes)))
print (len(classes), "classes", classes)

91 classes ['alamat_untar', 'alumni_terkenal', 'alur_pendaftaran', 'asrama_mahasiswa', 'beasiswa_berprestasi', 'beasiswa_untar', 'berdirinya_untar', 'biaya_kuliah', 'cara_membayar_spp', 'dispensasi_biaya', 'dokumen_yang_diperlukan', 'dukungan_kompetisi', 'event_job_fair', 'fakultas_untar', 'farewell', 'fasilitas_ibadah', 'fasilitas_kamarmandi', 'fasilitas_kesehatan', 'fasilitas_layanan_konseling', 'fasilitas_olahraga', 'fasilitas_parkir', 'fasilitas_perpustakaan', 'fasilitas_untar', 'fasilitas_wifi', 'greeting', 'hasil_seleksi', 'hubungan_dunia_industri', 'identitas_untar', 'jadwal_kuliah', 'jadwal_ujian', 'jadwal_wisuda', 'jumlah_mahasiswa', 'kantin_untar', 'kelebihan_untar', 'kerjasama_beasiswa', 'kerjasama_magang', 'kerjasama_rekrutmen', 'kompetisi_internal', 'komponen_biaya', 'kualitas_laboratorium', 'kualitas_ruang_kelas', 'kurikulum_universitas', 'libur_semester', 'magang_kurikulum', 'menghadapi_dunia_kerja', 'nilai_kuliah', 'organisasi_daftar', 'organisasi_minatdanbakat', 'pasca

## **Tokenization (Tokenisasi)**

In [9]:
# Tokenize the data (Tokenisasi Data)
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['patterns'])
train = tokenizer.texts_to_sequences(data['patterns'])
train

[[697],
 [513, 698],
 [699],
 [513],
 [342],
 [700],
 [701],
 [702],
 [703],
 [704],
 [705],
 [438],
 [234, 438],
 [706],
 [707],
 [439],
 [234, 439],
 [514],
 [440, 514],
 [441],
 [234, 441],
 [515],
 [440, 515],
 [442],
 [234, 442],
 [342, 234, 438],
 [342, 234, 439],
 [342, 234, 441],
 [342, 234, 442],
 [4, 84, 343],
 [48, 343],
 [344, 48],
 [344, 48, 23],
 [343, 4],
 [708, 343, 84],
 [48, 709],
 [343, 10, 11, 344, 48, 23],
 [48, 10, 443, 344],
 [344, 710, 124, 48],
 [516],
 [234, 283],
 [711, 517],
 [712],
 [713],
 [714],
 [715],
 [716],
 [440, 516],
 [188, 518],
 [188, 518, 307],
 [717],
 [718],
 [719, 517],
 [720, 167],
 [519],
 [444],
 [444, 261],
 [444, 721, 722],
 [519, 308, 125],
 [4, 19, 21, 120, 10, 235, 2, 8, 7],
 [4, 19, 59, 2, 8, 7],
 [445, 4, 22, 10, 6, 2, 1],
 [59, 4, 10, 235, 2, 9, 1],
 [4, 22, 151, 21, 120, 2, 8, 7],
 [21, 120, 2, 1, 6, 4, 22],
 [4, 59, 10, 60, 2, 1],
 [54, 14, 59, 4, 22, 10, 6, 2, 8, 7],
 [4, 22, 151, 59, 2, 1],
 [59, 46, 22, 10, 16, 520, 2, 8, 7],


## **Padding**


In [10]:
# Melakukan proses padding pada data
x_train = pad_sequences(train)
# Menampilkan hasil padding
print(x_train)

[[  0   0   0 ...   0   0 697]
 [  0   0   0 ...   0 513 698]
 [  0   0   0 ...   0   0 699]
 ...
 [  0   0   0 ...  81   1 148]
 [  0   0   0 ...  81   1  48]
 [  0   0   0 ... 976   2   1]]


Hasil setelah padding adalah setiap sequence memiliki panjang yang sama. Padding dapat melakukan ini dengan menambahkan 0 secara default pada awal sequence yang lebih pendek.

## **Encoding Text**

In [11]:
# Melakukan konversi data label tags dengan encoding
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])
print(y_train)

[24 24 24 ... 88 88 88]


In [12]:
# Melihat hasil input pada data teks
input_shape = x_train.shape[1]
print(input_shape)

14


In [13]:
# Melakukan definisi tiap kalimat dan kata pada data teks
vocabulary = len(tokenizer.word_index)
print("number of unique words : ", vocabulary)

# Melakukan pemeriksaan pada data output label teks
output_length = le.classes_.shape[0]
print("output length: ", output_length)

number of unique words :  976
output length:  91


**Input length** dan **output length** terlihat sangat jelas hasilnya. Mereka adalah untuk bentuk input dan bentuk output dari data train atau latih yang akan diproses pada algoritma LSTM yang akan dilatih.

**Vocabulary Size** adalah untuk lapisan penyematan untuk membuat representasi vektor unik untuk setiap kata.

## **Save Model Words & Classes**

Setelah dilakukan pemrosesan teks yang dilakukan lima tahap maka kita bisa simpan model pemrosesan teks tersebut dengan menggunakan format pickle.

Hal ini biasanya digunakan untuk membuat hubungan model yang telah dilatih dengan model pemrosesan teks.

In [14]:
# Simpan hasil pemrosesan teks dengan menggunakan pickle
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

## **Save Label Encoder & Tokenizer**

In [15]:
pickle.dump(le, open('le.pkl','wb'))
pickle.dump(tokenizer, open('tokenizers.pkl','wb'))

In [16]:
# Creating the model (Membuat Modelling)
i = Input(shape=(input_shape,)) # Layer Input
x = Embedding(vocabulary+1,10)(i) # Layer Embedding
x = LSTM(10, return_sequences=True, recurrent_dropout=0.2)(x) # Layer Long Short Term Memory
x = Flatten()(x) # Layer Flatten
x = Dense(output_length, activation="softmax")(x) # Layer Dense
model  = Model(i,x) # Model yang telah disusun dari layer Input sampai layer Output

# Compiling the model (Kompilasi Model)
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [17]:
# Visualization Plot Architecture Model (Visualisasi Plot Arsitektur Model)
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [18]:
# Menampilkan parameter pada model LSTM
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 14, 10)         │         9,770 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 14, 10)         │           840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 140)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 91)             │        12,831 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,441 (91.57 KB)

 Trainable params: 23,441 (91.57 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Training the model (Melatih model data sampai 300 kali)
train = model.fit(x_train, y_train, epochs=200)

Epoch 1/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0134 - loss: 4.5091
Epoch 2/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0349 - loss: 4.4810
Epoch 3/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0511 - loss: 4.4004
Epoch 4/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1029 - loss: 4.1890
Epoch 5/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1256 - loss: 3.9386
Epoch 6/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1672 - loss: 3.6787
Epoch 7/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2218 - loss: 3.3779
Epoch 8/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2611 - loss: 3.1733
Epoch 9/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3344 - loss: 2.9019
Epoch 10/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3800 - loss: 2.6892
Epoch 11/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4735 - loss: 2.4978
Epoch 12/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy:

# **Save The Model (Simpan Model)**


In [20]:
# Simpan model dalam bentuk format file .h5 atau .pkl (pickle)
model.save('model_lstm.h5')

print('Model Created Successfully!')

Model Created Successfully!
